In [47]:
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')
warnings.filterwarnings('ignore', category=DeprecationWarning)

import pandas as pd
pd.options.display.max_columns = 100

from matplotlib import pyplot as plt
import numpy as np

import seaborn as sns
sns.set(rc={'figure.figsize':(12,9)})
import pylab as plot
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss,accuracy_score
import pickle
import time

In [48]:
from xgboost import XGBClassifier
import xgboost as xgb

In [49]:
#for scaling
from sklearn.preprocessing import StandardScaler

Importing training dataset

In [108]:
data = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')


In [109]:
print(data.shape)

(867873, 10)


In [110]:
data.columns

Index(['Dates', 'Category', 'Descript', 'DayOfWeek', 'PdDistrict',
       'Resolution', 'Address', 'X', 'Y', 'Id'],
      dtype='object')


Splitting the "Dates" into date and time

In [111]:
df=data.head(100000)
datetime=df.Dates.str.split(pat=" ",expand=True)
datetime.columns=['Date','Time']
#datetime

#------------test data-------------

df_test=test
datetime_test=df_test.Dates.str.split(pat=" ",expand=True)
datetime_test.columns=['Date','Time']

In [112]:
Date=datetime.Date.str.split(pat="-",expand=True)
Date.columns=['Year','Month','Day']

Time=datetime.Time.str.split(pat=":",expand=True)
Time.columns=['Hour','Minute','Second']

#------------test data-------------

Date_test=datetime_test.Date.str.split(pat="-",expand=True)
Date_test.columns=['Year','Month','Day']

Time_test=datetime_test.Time.str.split(pat=":",expand=True)
Time_test.columns=['Hour','Minute','Second']

Dropping the column of "Dates" and appending the dataframe "datetime"

In [113]:
df=pd.concat([df,Date,Time],axis=1)
#df

#-----------test data------------------

df_test=pd.concat([df_test,Date_test,Time_test],axis=1)


In [114]:
df=df.drop(labels=['Dates'],axis=1)

#-----------test data------------------

df_test=df_test.drop(labels=['Dates'],axis=1)


# #Encoding

In [115]:
df.columns

Index(['Category', 'Descript', 'DayOfWeek', 'PdDistrict', 'Resolution',
       'Address', 'X', 'Y', 'Id', 'Year', 'Month', 'Day', 'Hour', 'Minute',
       'Second'],
      dtype='object')

LabelEncoding of "DayOfWeek" and "PdDistrict"

In [116]:
le = preprocessing.LabelEncoder()

In [117]:
#y=pd.get_dummies(df.Category,columns=['Category'],prefix=" ",prefix_sep=" ",drop_first=True,)

le_res=le.fit_transform(df['Category'])
y=pd.DataFrame(le_res)
y.columns=['Category']
#y

# Rotation 60 degree

In [118]:
df["rot60_X"]=(0.5) * df["Y"] + (1.732/2) * df["X"]
df["rot60_Y"]=0.5 * df["Y"] - (1.732/2) * df["X"]



df_test["rot60_X"]=(0.5) * df_test["Y"] + (1.732/2) * df_test["X"]
df_test["rot60_Y"]=0.5 * df_test["Y"] - (1.732/2) * df_test["X"]

In [119]:
df["radial60"]=np.sqrt(np.power(df['rot60_X'],2) + np.power(df['rot60_Y'],2))

df_test["radial60"]=np.sqrt(np.power(df_test['rot60_X'],2) + np.power(df_test['rot60_Y'],2))

# Month clubbing

In [120]:
df=df.drop(labels='rot60_X',axis=1)

df_test=df_test.drop(labels='rot60_X',axis=1)

In [121]:
df=df.drop(labels='rot60_Y',axis=1)

df_test=df_test.drop(labels='rot60_Y',axis=1)

In [122]:
df=df.drop(labels='Second',axis=1)

df_test=df_test.drop(labels='Second',axis=1)

# Minute Clubbing

In [123]:
df['Minute']=df['Minute'].apply(lambda x:int(x))
df['Minute']=df['Minute'].apply(lambda x : 'low' if x <31 else 'high')

df_test['Minute']=df_test['Minute'].apply(lambda x:int(x))
df_test['Minute']=df_test['Minute'].apply(lambda x : 'low' if x <31 else 'high')


# DayOfWeek

In [124]:
df['DayOfWeek']= df['DayOfWeek'].apply(lambda x : 'WeekHigh' if x in ('Wednesday','Friday') else ('WeekMed' if x in ('Tuesday','Thursday','Saturday') else 'WeekLow'))


df_test['DayOfWeek']= df_test['DayOfWeek'].apply(lambda x : 'WeekHigh' if x in ('Wednesday','Friday') else ('WeekMed' if x in ('Tuesday','Thursday','Saturday') else 'WeekLow'))


# Address Update

In [125]:
df['Intersection']=df['Address'].apply(lambda x : 1 if '/' in x else 0)
df['Block']=df['Address'].apply(lambda x : 1 if 'Block' in x else 0)
df_test['Intersection']=df_test['Address'].apply(lambda x : 1 if '/' in x else 0)
df_test['Block']=df_test['Address'].apply(lambda x : 1 if 'Block' in x else 0)

In [126]:
address=pd.DataFrame(df['Address'],columns=['Address'])
address=address.Address.str.split(pat=" /",expand=True )

address.columns=['Address','Intr2']

address=address.Address.str.split(pat=" /",expand=True )
address.columns=['Address']

string=address.iloc[:,0]
string=string.str.strip()

address_fram=string.to_frame()
temp=address_fram['Address'].astype(str).str[-2:]

address=temp.to_frame()

address['Address']=address['Address'].apply(lambda x :( x if x in ("ST","AV","LN","DR","BL","HY","CT","RD","PL","PZ","TR","AL","CR","WK","EX","RW") else (("I-80" if x in ("80") else ("HWY" if x in ("WY") else ("WAY" if x in ("AY") else ("TER" if x in ("ER") else ("ALMS" if x in ("MS") else ("MAR" if x in ("AR") else ("PARK" if x in ("RK") else ("STWY" if x in ("WY") else ("VIA" if x in ("NO") else ("BLOCK")))))))))))))
df=df.drop(labels=['Address'],axis=1)
df=pd.concat([address,df],axis=1)



# Address splitting

In [127]:
address_test=pd.DataFrame(df_test['Address'],columns=['Address'])
address_test=address_test.Address.str.split(pat=" /",expand=True )

address_test.columns=['Address','Intr2']

address_test=address_test.Address.str.split(pat=" /",expand=True )
address_test.columns=['Address']

string_test=address_test.iloc[:,0]
string_test=string_test.str.strip()

address_fram_test=string_test.to_frame()
temp_test=address_fram_test['Address'].astype(str).str[-2:]

address_test=temp_test.to_frame()

address_test['Address']=address_test['Address'].apply(lambda x :( x if x in ("ST","AV","LN","DR","BL","HY","CT","RD","PL","PZ","TR","AL","CR","WK","EX","RW") else (("I-80" if x in ("80") else ("HWY" if x in ("WY") else ("WAY" if x in ("AY") else ("TER" if x in ("ER") else ("ALMS" if x in ("MS") else ("MAR" if x in ("AR") else ("PARK" if x in ("RK") else ("STWY" if x in ("WY") else ("VIA" if x in ("NO") else ("BLOCK")))))))))))))
df_test=df_test.drop(labels=['Address'],axis=1)
df_test=pd.concat([address_test,df_test],axis=1)

In [128]:
Id=df['Id']
df=df.drop(['Descript','Resolution','Id'],axis=1)

#----------test data---------

Id_test=df_test['Id']
df_test=df_test.drop(['Descript','Resolution','Id'],axis=1)

In [129]:
df_test.columns

Index(['Address', 'DayOfWeek', 'PdDistrict', 'X', 'Y', 'Year', 'Month', 'Day',
       'Hour', 'Minute', 'radial60', 'Intersection', 'Block'],
      dtype='object')

In [130]:
data['PdDistrict'].unique()

array(['MISSION', 'SOUTHERN', 'BAYVIEW', 'CENTRAL', 'INGLESIDE',
       'NORTHERN', 'RICHMOND', 'TARAVAL', 'TENDERLOIN', 'PARK'],
      dtype=object)

# Label Encoding

In [131]:

df.columns

Index(['Address', 'Category', 'DayOfWeek', 'PdDistrict', 'X', 'Y', 'Year',
       'Month', 'Day', 'Hour', 'Minute', 'radial60', 'Intersection', 'Block'],
      dtype='object')

In [132]:
df_test.columns

Index(['Address', 'DayOfWeek', 'PdDistrict', 'X', 'Y', 'Year', 'Month', 'Day',
       'Hour', 'Minute', 'radial60', 'Intersection', 'Block'],
      dtype='object')

In [133]:
le = preprocessing.LabelEncoder()

Label Encoding :DayOfWeek

In [134]:

#le = preprocessing.LabelEncoder()
le_res=le.fit_transform(df['DayOfWeek'])
Day=pd.DataFrame(le_res)
Day.columns=['DayOfWeek']
df=df.drop(labels=['DayOfWeek'],axis=1)
df=pd.concat([Day,df],axis=1)

#----------test data----------

le_res_test=le.fit_transform(df_test['DayOfWeek'])
Day_test=pd.DataFrame(le_res_test)
Day_test.columns=['DayOfWeek']
df_test=df_test.drop(labels=['DayOfWeek'],axis=1)
df_test=pd.concat([Day_test,df_test],axis=1)

Label Encoding: PdDistrict

In [135]:

le_res=le.fit_transform(df['PdDistrict'])
District=pd.DataFrame(le_res)

#District=pd.get_dummies(df['PdDistrict'],drop_first=True)
District.columns=['District']
df=df.drop(labels=['PdDistrict'],axis=1)
df=pd.concat([District,df],axis=1)


le_res_test=le.fit_transform(df_test['PdDistrict'])
District_test=pd.DataFrame(le_res_test)

#District=pd.get_dummies(df['PdDistrict'],drop_first=True)
District_test.columns=['District']
df_test=df_test.drop(labels=['PdDistrict'],axis=1)
df_test=pd.concat([District_test,df_test],axis=1)


Label encoding: Year

In [136]:

#le = preprocessing.LabelEncoder()
le_res=le.fit_transform(df['Year'])
Year=pd.DataFrame(le_res)

#District=pd.get_dummies(df['PdDistrict'],drop_first=True)
Year.columns=['Year']
df=df.drop(labels=['Year'],axis=1)
df=pd.concat([Year,df],axis=1)


#le = preprocessing.LabelEncoder()
le_res_test=le.fit_transform(df_test['Year'])
Year_test=pd.DataFrame(le_res_test)

#District=pd.get_dummies(df['PdDistrict'],drop_first=True)
Year_test.columns=['Year']
df_test=df_test.drop(labels=['Year'],axis=1)
df_test=pd.concat([Year_test,df_test],axis=1)


Label encoding: Month

In [137]:

#le = preprocessing.LabelEncoder()
le_res=le.fit_transform(df['Month'])
Month=pd.DataFrame(le_res)

#District=pd.get_dummies(df['PdDistrict'],drop_first=True)
Month.columns=['Month']
df=df.drop(labels=['Month'],axis=1)
df=pd.concat([Month,df],axis=1)


#le = preprocessing.LabelEncoder()
le_res_test=le.fit_transform(df_test['Month'])
Month_test=pd.DataFrame(le_res_test)

#District=pd.get_dummies(df['PdDistrict'],drop_first=True)
Month_test.columns=['Month']
df_test=df_test.drop(labels=['Month'],axis=1)
df_test=pd.concat([Month_test,df_test],axis=1)


Label encoding: Day

In [138]:

#le = preprocessing.LabelEncoder()
le_res=le.fit_transform(df['Day'])
Day=pd.DataFrame(le_res)

#District=pd.get_dummies(df['PdDistrict'],drop_first=True)
Day.columns=['Day']
df=df.drop(labels=['Day'],axis=1)
df=pd.concat([Day,df],axis=1)


#le = preprocessing.LabelEncoder()
le_res_test=le.fit_transform(df_test['Day'])
Day_test=pd.DataFrame(le_res_test)

#District=pd.get_dummies(df['PdDistrict'],drop_first=True)
Day_test.columns=['Day']
df_test=df_test.drop(labels=['Day'],axis=1)
df_test=pd.concat([Day_test,df_test],axis=1)


Label encoding: Hour

In [139]:

#le = preprocessing.LabelEncoder()
le_res=le.fit_transform(df['Hour'])
Hour=pd.DataFrame(le_res)

#District=pd.get_dummies(df['PdDistrict'],drop_first=True)
Hour.columns=['Hour']
df=df.drop(labels=['Hour'],axis=1)
df=pd.concat([Hour,df],axis=1)


#le = preprocessing.LabelEncoder()
le_res_test=le.fit_transform(df_test['Hour'])
Hour_test=pd.DataFrame(le_res_test)

#District=pd.get_dummies(df['PdDistrict'],drop_first=True)
Hour_test.columns=['Hour']
df_test=df_test.drop(labels=['Hour'],axis=1)
df_test=pd.concat([Hour_test,df_test],axis=1)


Label encoding: Minute

In [140]:

le_res=le.fit_transform(df['Minute'])
Minute=pd.DataFrame(le_res)

#District=pd.get_dummies(df['PdDistrict'],drop_first=True)
Minute.columns=['Minute']
df=df.drop(labels=['Minute'],axis=1)
df=pd.concat([Minute,df],axis=1)


le_res_test=le.fit_transform(df_test['Minute'])
Minute_test=pd.DataFrame(le_res_test)

#District=pd.get_dummies(df['PdDistrict'],drop_first=True)
Minute_test.columns=['Minute']
df_test=df_test.drop(labels=['Minute'],axis=1)
df_test=pd.concat([Minute_test,df_test],axis=1)



#label encode street suffix

In [141]:
le_res=le.fit_transform(df['Category'])
cat=pd.DataFrame(le_res)
cat.columns=['Category']
df=df.drop(labels=['Category'],axis=1)
df=pd.concat([cat,df],axis=1)

df.columns

Index(['Category', 'Minute', 'Hour', 'Day', 'Month', 'Year', 'District',
       'DayOfWeek', 'Address', 'X', 'Y', 'radial60', 'Intersection', 'Block'],
      dtype='object')

In [142]:

le_res=le.fit_transform(df['Address'])
Address=pd.DataFrame(le_res)
Address.columns=['Address']
df=df.drop(labels=['Address'],axis=1)
df=pd.concat([Address,df],axis=1)
le_res=le.fit_transform(df_test['Address'])
Address_test=pd.DataFrame(le_res)
Address_test.columns=['Address']
df_test=df_test.drop(labels=['Address'],axis=1)
df_test=pd.concat([Address_test,df_test],axis=1)

In [143]:
df

,Address,Category,Minute,Hour,Day,Month,Year,District,DayOfWeek,X,Y,radial60,Intersection,Block
0,17,27,0,17,27,5,10,3,0,-122.419331,37.762264,152.287276,0,1
1,17,17,0,2,18,1,1,7,2,-122.411340,37.781271,152.279998,0,1
2,17,13,1,0,13,10,4,7,0,-122.403405,37.775421,152.269705,0,1
3,2,24,1,18,26,11,4,0,2,-122.404715,37.730161,152.265674,0,1
4,17,21,1,17,8,8,9,7,1,-122.405239,37.785265,152.273137,1,0
5,17,34,1,6,13,4,11,3,0,-122.418405,37.766760,152.286716,1,0
6,12,4,0,23,8,7,3,1,0,-122.404651,37.790442,152.273071,0,1
7,17,30,0,12,15,10,9,2,0,-122.410871,37.737526,152.274009,0,1
8,17,21,0,23,4,1,4,0,1,-122.405911,37.732964,152.267463,1,0
9,17,7,0,7,6,1,3,4,2,-122.417710,37.784236,152.288047,1,0


In [ ]:
df=df[[ 'District', 'X', 'Y', 'Year','Hour','Minute','Address', 'radial60', 'Intersection','Block']]


df_test=df_test[[ 'District', 'X', 'Y', 'Year','Hour','Address',
       'Month', 'Day', 'radial60', 'Intersection','Block']]

In [207]:
df['Address'].nunique()

22

In [208]:
df_test['Address'].nunique()

18

In [209]:
df_test.columns

Index(['District', 'X', 'Y', 'Year', 'Hour', 'Address', 'Month', 'Day',
       'radial60', 'Intersection', 'Block'],
      dtype='object')

In [210]:
A=df.corr()
A

,District,X,Y,Year,Hour,Minute,Address,radial60,Intersection,Block
District,1.000000,-0.000747,-0.004491,0.007874,0.007614,-0.021603,0.079709,0.157222,-0.019354,0.019354
X,-0.000747,1.000000,0.028295,0.000267,-0.000298,0.001300,0.004086,0.000529,-0.001023,0.001023
Y,-0.004491,0.028295,1.000000,-0.003798,0.006727,-0.004175,0.002593,0.003808,-0.000840,0.000840
Year,0.007874,0.000267,-0.003798,1.000000,-0.009618,-0.015420,0.009450,-0.008733,-0.026512,0.026512
Hour,0.007614,-0.000298,0.006727,-0.009618,1.000000,0.003234,0.003987,-0.003431,0.048190,-0.048190
Minute,-0.021603,0.001300,-0.004175,-0.015420,0.003234,1.000000,-0.022630,0.027744,-0.040247,0.040247
Address,0.079709,0.004086,0.002593,0.009450,0.003987,-0.022630,1.000000,-0.128854,0.053993,-0.053993
radial60,0.157222,0.000529,0.003808,-0.008733,-0.003431,0.027744,-0.128854,1.000000,-0.005568,0.005568
Intersection,-0.019354,-0.001023,-0.000840,-0.026512,0.048190,-0.040247,0.053993,-0.005568,1.000000,-1.000000
Block,0.019354,0.001023,0.000840,0.026512,-0.048190,0.040247,-0.053993,0.005568,-1.000000,1.000000


# Splitting into test and train


In [211]:
X=df

In [212]:
#splitting the dataset into the training set and test set
#from sklearn import model_selection as ms

X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.33,random_state=0)

# XGBoost Model

In [213]:
X_train.head()

,District,X,Y,Year,Hour,Minute,Address,radial60,Intersection,Block
32684,7,0.0,0.0,10,1,1,17,152.277816,0,1
77206,9,0.0,0.0,2,20,1,17,152.283159,0,1
17196,3,0.0,0.0,1,22,1,17,152.276075,1,0
94613,5,0.0,0.0,5,8,1,17,152.322495,0,1
61769,0,0.0,0.0,11,8,1,17,152.265608,0,1


In [214]:
model = xgb.XGBClassifier(objective='multi:softprob')
model.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='multi:softprob', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [215]:
y_pred=model.predict_proba(X_test)

In [216]:
print (log_loss(y_test,y_pred))

2.461279888905359


In [218]:
import lightgbm as lgb
train_data=lgb.Dataset(X_train,label=y_train)
#define parameters

model= lgb.train(train_data, 100) 
y_pred=model.predict_proba(x_test)
print (log_loss(y_test,y_pred))

ModuleNotFoundError: No module named 'lightgbm'